In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
import numpy as np
import tqdm
import matplotlib.pyplot as plt
import pathlib
from sklearn.cluster import KMeans
import cv2

import common.loss_utils as loss_utils
import common.climatehack_dataset as climatehack_dataset 


In [3]:
p = pathlib.Path('data/test.npz')
f = np.load(p)
times = f['times']
data = f['data']

In [4]:
times.shape, data.shape

((2189,), (2189, 325, 400))

In [14]:
INPUT_STEPS=4

def check_times(tstart, tend):
    return int((tend - tstart) / np.timedelta64(1, 'm')) == 175

class CustomTensorDataset(torch.utils.data.Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, times, data, random_state=7):
        self.times = times
        self.data = data
        self.generator = np.random.RandomState(random_state)
        
        
    def _get_crop(self, data):
        # roughly over the mainland UK
        rand_x = self.generator.randint(0, data.shape[2] - 128)
        rand_y = self.generator.randint(0, data.shape[1] - 128)
        # make a data selection
        return data[:, rand_y : rand_y + 128, rand_x : rand_x + 128]

    def __getitem__(self, index):
        tend = self.times[index + 35]
        tstart = self.times[index]
        if not check_times(tstart, tend):
            return self.__getitem__((index + 35) % len(self))
        all_data = self.data[index:index+INPUT_STEPS+24]
        all_data = self._get_crop(all_data)
        x = all_data[:INPUT_STEPS]
        # grab center crop
        y = all_data[INPUT_STEPS:,32:96,32:96]
        return x, y

    def __len__(self):
        return len(self.times) - 35

ds = CustomTensorDataset(times, data)

In [19]:
def warp_flow(img, flow):
    h, w = flow.shape[:2]
    flow = -flow
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]
    res = cv2.remap(img, flow, None, cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
    return res

def get_mssim(y, x, average=True):
    if len(y.shape) == 2:
        # we need to specify a number of channels. This sets it to 1 and calls the function again
        return get_mssim(np.expand_dims(y, 0), np.expand_dims(x, 0))
    if len(y.shape) == 4:
        y = y.reshape(-1, y.shape[2], y.shape[3])
        x = x.reshape(-1, x.shape[2], x.shape[3])
    assert len(y.shape) == 3
    if average:
        mssim_loss = loss_utils.MS_SSIMLoss(data_range=1023, channels=y.shape[0])
        yd = np.expand_dims(y, 0) # /255 * (MAX - MIN) + MIN
        xd = np.expand_dims(x, 0) # /255 * (MAX - MIN) + MIN
        return 1.0 - mssim_loss(
            torch.Tensor(yd),
            torch.Tensor(xd),
        ).item()
    else:
        ret = []
        for i in range(y.shape[0]):
            score = get_mssim(y[i], x[i])
            ret.append(score)
        return ret

In [20]:
# combine classical classical
combined_optical_flow_scores = []
params = {
    'iterations': 1,
    'levels': 10,
    'poly_n': 2,
    'poly_sigma': 1.3358462139572997,
    'pyr_scale': 0.13581496550086464,
    'winsize': 3
}
test_params = {
    'pyr_scale': 0.5,
    'levels': 2,
    'winsize': 40,
    'iterations': 3, 
    'poly_n': 5,
    'poly_sigma': 0.7
}
forecast = 6
for i in tqdm.tqdm(range(len(ds))):
    x, y = ds[i]
    preds = np.zeros(y.shape, dtype=np.float32)
    cur_feol = x[-1].astype(np.float32)
    cur_beol = x[-1].astype(np.float32)
    flow_feol = cv2.calcOpticalFlowFarneback(
        prev=x[-2],
        next=x[-1],
        flow=None,
        **test_params,
        flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN,
    )
    flow_beol = cv2.calcOpticalFlowFarneback(
        prev=x[-2],
        next=x[-1],
        flow=None,
        **params,
        flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN,
    )
    for i in range(forecast):
        cur_feol = warp_flow(cur_feol, flow_feol)
        cur_beol = warp_flow(cur_beol, flow_beol)
        cur_beol = cv2.blur(cur_beol, (3, 3))
        preds[i] = cur_feol[32:96,32:96]
    for i in range(forecast, 24):
        cur_beol = warp_flow(cur_beol, flow_beol)
        cur_beol = cv2.blur(cur_beol, (3, 3))
        preds[i] = cur_beol[32:96,32:96]

    score = get_mssim(preds, y, average=False)
    combined_optical_flow_scores.append(score)

100%|██████████| 2154/2154 [01:45<00:00, 20.43it/s]


In [26]:
combined_optical_flow_scores = np.array(combined_optical_flow_scores)
combined_step_means = np.mean(combined_optical_flow_scores, axis=0)

In [25]:
np.mean(combined_optical_flow_scores)

0.7243027332103665

In [27]:
combined_step_means

array([0.96158783, 0.91150104, 0.86790412, 0.83004581, 0.79700563,
       0.76777844, 0.74281813, 0.72972036, 0.71817764, 0.70856177,
       0.69985263, 0.69224906, 0.68568141, 0.67974315, 0.67498827,
       0.67027487, 0.6661345 , 0.66238309, 0.65867075, 0.65588156,
       0.65305837, 0.65129956, 0.64980159, 0.64814601])

In [5]:
combined_step_means = np.array([0.96158783, 0.91150104, 0.86790412, 0.83004581, 0.79700563,
       0.76777844, 0.74281813, 0.72972036, 0.71817764, 0.70856177,
       0.69985263, 0.69224906, 0.68568141, 0.67974315, 0.67498827,
       0.67027487, 0.6661345 , 0.66238309, 0.65867075, 0.65588156,
       0.65305837, 0.65129956, 0.64980159, 0.64814601])

In [4]:
dgmr = np.array([0.83147662, 0.7995316 , 0.769489  , 0.74837667, 0.73493329,
       0.72543577, 0.71969761, 0.71617059, 0.71314882, 0.70988742,
       0.70612073, 0.70189681, 0.69773933, 0.69393087, 0.69029389,
       0.68646716, 0.68398141, 0.68153732, 0.67900101, 0.67651455,
       0.67394017, 0.67143976, 0.66915577, 0.66786966])

In [6]:
choice = 10
np.array(combined_step_means[:choice].tolist() + dgmr[choice:].tolist()).mean()

0.7339578837499999

In [7]:
choice = 6
np.array(combined_step_means[:choice].tolist() + dgmr[choice:].tolist()).mean()

0.73227565625